In [1]:
import cv2
import numpy as np
from PIL import Image

import importlib

import py_vncorenlp
import torch
from transformers import AutoModel, AutoTokenizer

import preprocess_img
import LineOCR
importlib.reload(LineOCR)

import pandas as pd

import graph
from torch_geometric.utils.convert import from_networkx
import torch_geometric
importlib.reload(graph)

import torch.nn as nn
from torch_geometric.nn import ChebConv, GCNConv
import torch.nn.functional as F

import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu").type


ID2LABEL_DICT = {
    0: "None",
    1: "sign_date",
    2: "diagnose",
    3:"hospital_name",
    4:"address",
    5:"age",
    6:"treatment",
    7:"patient_name",
    8:"admission_date",
    9:"discharge_date",
    10:"gender",
    11:"document_type",
    12:"department",
    13:"note",
    14:"BHYT",
}

INFO 2022-10-19 15:23:52,669 utils.py:147] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [2]:
#Load Text Segmentation engine
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir=r'D:\PPYCode\OCR\ppy_paddle\weights\nlp\vncorenlp')
phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
%cd ../../../

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


D:\PPYCode\OCR\ppy_paddle


In [2]:
pwd

'd:\\PPYCode\\OCR\\ppy_paddle'

In [3]:
#Load all models
preprocessImage = preprocess_img.PreprocessImage()
lineDetAndOCR = LineOCR.ProcessImage()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'ppocr/postprocess/pse_postprocess/pse'

In [ ]:
test_img = cv2.imread(r"imgs_test\705.jpeg")
test_img = preprocessImage(test_img)
# lines, t = lineDetInfer.line_det_infer(test_img)

In [ ]:
# image = test_img.copy()
# for line in lines:
#     pts = line.astype(int)
#     pts = pts.reshape((-1, 1, 2))
    
#     isClosed = True
    
#     # Blue color in BGR
#     color = (255, 0, 0)
    
#     # Line thickness of 2 px
#     thickness = 2
    
#     image = cv2.polylines(image, [pts],
#                         isClosed, color, thickness)

# display(Image.fromarray(image))

In [5]:
result = process_engine.begin_recognize_text(test_img)

33


In [ ]:
for iter in range(len(result["croped_pil_img"])):
    line_img = result["croped_pil_img"][iter]
    text = result["text"][iter]
    ocr_score = result["ocr_score"][iter]
    display(line_img)
    print(f"{text} \t {ocr_score}")
    print("---------------------------------")

In [7]:
list_xmin = []
list_ymin = []
list_xmax = []
list_ymax = []
list_Object = []

for iter in range(len(result["bbox"])):
    xmin, ymin, xmax, ymax = result["bbox"][iter]
    text = result["text"][iter]
    list_xmin.append(xmin)
    list_xmax.append(xmax)
    list_ymin.append(ymin)
    list_ymax.append(ymax)
    list_Object.append(text)
single_df = {
    "xmin": list_xmin,
    "xmax": list_xmax,
    "ymin": list_ymin,
    "ymax": list_ymax,
    "Object": list_Object,
    "labels": ["None"]*len(list_Object)
}

In [8]:
single_df = pd.DataFrame.from_dict(single_df)

In [9]:
single_df.to_excel("temp_data/temp_gcn/csv/temp.xlsx")
cv2.imwrite("temp_data/temp_gcn/img/temp.jpg", test_img)

True

In [11]:
def get_sentence_features(sentence):
    segmented_sentence = rdrsegmenter.word_segment(sentence)
    tokenized_sentence = torch.tensor([tokenizer.encode(segmented_sentence[0])])
    features = []
    with torch.no_grad():
        features = phobert(tokenized_sentence)
    return features

def make_sent_bert_features(text):
    features = get_sentence_features(text)
    return features[1][0].cpu().numpy()

In [14]:
file = 'temp'
connect = graph.Grapher(file)
G,result, df = connect.graph_formation(export_graph=False)
df = connect.relative_distance(export_document_graph = False)
individual_data = from_networkx(G)


In [15]:
feature_cols = [
            "rd_b",
            "rd_r",
            "rd_t",
            "rd_l",
            "line_number",
            "n_upper",
            "n_alpha",
            "n_spaces",
            "n_numeric",
            "n_special",
        ]

In [16]:
def preprocess_inference_data(df, individual_data):
        text_features = []
        for _, row in df.iterrows():
            text_features.append(make_sent_bert_features(row["Object"]))
        text_features = np.asarray(text_features, dtype=np.float32)

        numeric_features = df[feature_cols].values.astype(np.float32)

        features = np.concatenate((numeric_features, text_features), axis=1)
        features = torch.tensor(features)

        for col in df.columns:
            try:
                df[col] = df[col].str.strip()
            except AttributeError as e:
                pass
        text = df["Object"].values
        individual_data.x = features

        individual_data.text = text

        test_list_of_graphs = []
        test_list_of_graphs.append(individual_data)

        test_one_data = ""
        test_one_data = torch_geometric.data.Batch.from_data_list(test_list_of_graphs)
        test_one_data.edge_attr = None
        return test_one_data

In [17]:

test_data = preprocess_inference_data(df, individual_data)
test_data.to(device)

DataBatch(edge_index=[2, 84], num_nodes=33, x=[33, 778], text=[1], batch=[33], ptr=[2])

In [18]:
class InvoiceGCN(nn.Module):
    def __init__(self, input_dim, chebnet=False, n_classes=5, dropout_rate=0.2, K=3):
        super().__init__()

        self.input_dim = input_dim
        self.n_classes = n_classes
        self.dropout_rate = dropout_rate

        if chebnet:
            self.conv1 = ChebConv(self.input_dim, 64, K=K)
            # self.conv2 = ChebConv(64, 32, K=K)
            self.conv3 = ChebConv(64, 64, K=K)
            self.conv4 = ChebConv(64, self.n_classes, K=K)
        else:
            self.conv1 = GCNConv(self.first_dim, 64, improved=True, cached=True)
            self.conv2 = GCNConv(64, 32, improved=True, cached=True)
            self.conv3 = GCNConv(32, 16, improved=True, cached=True)
            self.conv4 = GCNConv(16, self.n_classes, improved=True, cached=True)

    def forward(self, data):
        # for transductive setting with full-batch update
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr

        x = F.dropout(F.relu(self.conv1(x, edge_index, edge_weight)), p=self.dropout_rate, training=self.training)
        # x = F.dropout(F.relu(self.conv2(x, edge_index, edge_weight)), p=self.dropout_rate, training=self.training)
        x = F.dropout(F.relu(self.conv3(x, edge_index, edge_weight)), p=self.dropout_rate, training=self.training)
        x = self.conv4(x, edge_index, edge_weight)

        return F.log_softmax(x, dim=1)

In [19]:
model = torch.load(r"D:\PPYCode\OCR\ppy_paddle\weights\gcn\GCN_221017.pth")
model.to(device)

InvoiceGCN(
  (conv1): ChebConv(778, 64, K=3, normalization=sym)
  (conv3): ChebConv(64, 64, K=3, normalization=sym)
  (conv4): ChebConv(64, 15, K=3, normalization=sym)
)

In [20]:
y_preds = model(test_data).max(dim=1)[1].cpu().numpy()

In [21]:
for idx, each in enumerate(y_preds):
    text = test_data.text[0][idx]
    pred_class = ID2LABEL_DICT[each]
    print(text + '\t' + pred_class)
    print("-----------------")


SỞ Y TẾ THANH HÓA	None
-----------------
CÔNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM	None
-----------------
MS:01/BV-01	None
-----------------
BỆNH VIỆN ĐA KHOA TỈNH	hospital_name
-----------------
Độc lập - Tự do - Hạnh phúc	None
-----------------
Số lưu trữ: 2206586	None
-----------------
Khoa Nội Thận - Tiết Niệu	department
-----------------
Mã Y tế: 38/280/2022	None
-----------------
GIẤY RA VIỆN	document_type
-----------------
- Họ tên người bệnh: NGUYỄN THỊ HỒNG MAI	patient_name
-----------------
Tuổi: 29	age
-----------------
Giới tính: Nữ	gender
-----------------
- Dân tộc: Kinh	None
-----------------
Nghề nghiệp: Khác	None
-----------------
- Mã số BHXH/Thẻ BHYT số:	None
-----------------
GD4383822386956	BHYT
-----------------
- Địa chỉ: Nhân Trạch, Xã Hoằng Đạo, Huyện Hoằng Hóa, Tỉnh Thanh Hóa	address
-----------------
- Vào viện lúc: 20 giờ 10 phút, ngày 19 tháng 01 năm 2022	admission_date
-----------------
- Ra viện lúc: 15 giờ 00 phút, ngày 27 tháng 01 năm 2022	discharge_date
-----